In [11]:
sys.path.append('..')
from config import Config

import json
import numpy as np # Для работы с данными 
import sys
import pymorphy2
import os

from tensorflow.keras import utils # Для работы с категориальными данными
from tensorflow.keras.preprocessing.text import Tokenizer # Методы для работы с текстами

In [13]:
JSON_DATA_DIR = '../../Satellites'

Все тексты true и false объединены в два больших текста;
Датасет - список из двух элементов-строк

In [14]:
# создаём два списка, в пустые элементы которых будем добавлять тексты
# в первом элементе будет храниться текст с ответом False, во втором - True
textClasses = ['', '']


# открываем файл с id эссе и ответами
with open(JSON_DATA_DIR + '/train/train_standart.json', 'r') as f_list:
  data = json.load(f_list)

  # проходимся по каждому "блоку" с эссе
  for i in range(len(data)):
    elem = data[i]

    with open(JSON_DATA_DIR + f'/train/essays/{elem["id"]}.json', 'r') as essay:
      file = json.load(essay)
      text = file['text']
      if elem['answer'] == False:
        textClasses[0] += text
        textClasses[0] += '#'
      else:
        textClasses[1] += text
        textClasses[1] += '#'

In [17]:
print(len(textClasses[0].split('.')))
print(len(textClasses[1].split('.')))

24526
6374


In [18]:
textClasses[0] = textClasses[0][:len(textClasses[1])]

In [24]:
print(len(textClasses[0]))
print(len(textClasses[1]))

print(textClasses[0][-50:])
print(textClasses[1][-50:])

texts_false = textClasses[0].split("#")
texts_true = textClasses[1].split("#")

print(len(texts_false))
print(len(texts_true))

654756
654756
е я могу на примере романа И. А. Гончарова «Обломо
рии и заключается противоречивость Раскольникова.#
222
201


In [29]:
test_border_false = len(texts_false)//10
test_border_true = len(texts_true)//10

val_border_false = len(texts_false)//20 + test_border_false
val_border_true = len(texts_true)//20 + test_border_true


valText = []
valText.append(' '.join(texts_false[:test_border_false]))
valText.append(' '.join(texts_true[:test_border_true]))

testText = []
testText.append(' '.join(texts_false[test_border_false:val_border_false]))
testText.append(' '.join(texts_true[test_border_true:val_border_false]))

trainText = []
trainText.append(' '.join(texts_false[val_border_false:]))
trainText.append(' '.join(texts_true[val_border_true:]))

In [30]:
print(len(testText[0]))
print(len(testText[1]))
print(len(valText[0]))
print(len(valText[1]))
print(len(trainText[0]))
print(len(trainText[1]))

33287
40826
73617
58625
547850
564659


Список параметров, которые будут меняться в течение генерации датасетов

In [31]:
numWordsList = [5000, 10000, 20000, "all"]
pymorphyList = ["", "PyMorphy"]
xLenList = [100, 300, 500]
stepList = [
    [25,  50,  100],
    [100, 150, 300],
    [100, 250, 500]
]
bowList = ["BOW", ""]

In [32]:
def getSetFromIndexes(wordIndexes, xLen, step):
  xText = []
  wordsLen = len(wordIndexes) # Считаем количество слов
  index = 0 # Задаем начальный индекс 

  while (index + xLen <= wordsLen): # Идём по всей длине вектора индексов
    xText.append(wordIndexes[index:index+xLen]) # "Откусываем" векторы длины xLen
    index += step # Смещаемся вперёд на step
    
  return xText

def createSetsMultiClasses(wordIndexes, xLen, step): # Функция принимает последовательность индексов, размер окна, шаг окна
  nClasses = len(wordIndexes) # Количество классов
  classesXSamples = []        # Здесь будет список размером "кол-во классов*кол-во окон в тексте*длину окна"
  for wI in wordIndexes:      # Для каждого текста выборки из последовательности индексов
    classesXSamples.append(getSetFromIndexes(wI, xLen, step))

  # Формируем один общий xSamples
  xSamples = []
  ySamples = []
  
  for t in range(nClasses):
    xT = classesXSamples[t]
    for i in range(len(xT)): # Перебираем каждое окно определенного класса
      xSamples.append(xT[i]) # Добавляем в общий список выборки
      ySamples.append(utils.to_categorical(t, nClasses)) # Добавляем соответствующий вектор класса

  xSamples = np.array(xSamples)
  ySamples = np.array(ySamples)

  return (xSamples, ySamples)

In [33]:
def text2Words(text):
  morph = pymorphy2.MorphAnalyzer() # Создаем экземпляр класса MorphAnalyzer
  words = text.split(' ') # Разделяем текст на пробелы
  words = " ".join([morph.parse(word)[0].normal_form for word in words]) #Переводим каждое слово в нормалную форму  
  # words = [morph.parse(word)[0].normal_form for word in words] #Переводим каждое слово в нормалную форму  
  return words # Возвращаем слова

In [8]:
# tokenizer = Tokenizer(num_words=20000,
#     filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff',
#     lower=True,
#     split=' ',
#     oov_token='unknown',
#     char_level=False)

# morphAnalyzer = pymorphy2.MorphAnalyzer
# nw_tokenizer = morphAnalyzer.iter_known_word_parses(tokenizer)
# print(list(nw_tokenizer))

In [9]:
# trainText[0] = text2Words(trainText[0])
# trainText[1] = text2Words(trainText[1])

# tokenizer = Tokenizer(num_words=10000,
#                 filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff',
#                 lower=True,
#                 split=' ',
#                 oov_token='unknown',
#                 char_level=False)

# tokenizer.fit_on_texts(trainText) # словарь частотности
# items = list(tokenizer.word_index.items()) # индексы слов

# print("Самые часто встречающиеся слова:")
# print(items[:10])
# print()
# print("Самые редко встречающиеся слова:")
# print(items[-10:])
# print()
# print("Размер словаря:", len(items))

In [34]:
labels = ['False', 'True']
labelsNum = len(labels)

for pymorphyState in pymorphyList:
    print(pymorphyState)
    
    if pymorphyState:
        for i in range(2):
            trainText[i] = text2Words(trainText[i])
            valText[i] = text2Words(valText[i])
            testText[i] = text2Words(testText[i])
        
    for numWords in numWordsList:
        print("numwords", numWords)
        tokenizer = Tokenizer(num_words=numWords,
            filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff',
            lower=True,
            split=' ',
            oov_token='unknown',
            char_level=False)
            
        if numWords == "all":
            tokenizer = Tokenizer(
                filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff',
                lower=True,
                split=' ',
                oov_token='unknown',
                char_level=False)
            
            tokenizer.fit_on_texts(trainText)
            items = list(tokenizer.word_index.items())
            numWords = len(items)
            
        tokenizer.fit_on_texts(trainText)

        trainWordIndexes = tokenizer.texts_to_sequences(trainText)
        valWordIndexes = tokenizer.texts_to_sequences(valText)
        testWordIndexes = tokenizer.texts_to_sequences(testText)

        for xLenIndex in range(len(xLenList)):
            print("xLenIndex", xLenIndex)
            for step in stepList[xLenIndex]:
                print("step", step)
                xLen = xLenList[xLenIndex]
                step = step

                xTrainId, yTrain = createSetsMultiClasses(trainWordIndexes, xLen, step)
                xValId, yVal = createSetsMultiClasses(valWordIndexes, xLen, step)
                xTestId, yTest = createSetsMultiClasses(testWordIndexes, xLen, step)

                for bowState in bowList:
                    print("bowState", bowState)
                    if bowState:
                        xTrain = tokenizer.sequences_to_matrix(xTrainId.tolist())
                        xVal = tokenizer.sequences_to_matrix(xValId.tolist())
                        xTest  = tokenizer.sequences_to_matrix(xTestId.tolist())
                    else:
                        xTrain = xTrainId
                        xVal = xValId
                        xTest = xTestId

                    np.savez_compressed(f"{Config.DATASETS_DIR}dataset_numWords_{numWords}_{pymorphyState}_xLen_{xLen}_step_{step}_{bowState}.npz", xTrain=xTrain, yTrain=yTrain, xVal=xVal, yVal=yVal, xTest=xTest, yTest=yTest)
            


numwords 5000
xLenIndex 0
step 25
bowState BOW
bowState 
step 50
bowState BOW
bowState 
step 100
bowState BOW
bowState 
xLenIndex 1
step 100
bowState BOW
bowState 
step 150
bowState BOW
bowState 
step 300
bowState BOW
bowState 
xLenIndex 2
step 100
bowState BOW
bowState 
step 250
bowState BOW
bowState 
step 500
bowState BOW
bowState 
numwords 10000
xLenIndex 0
step 25
bowState BOW
bowState 
step 50
bowState BOW
bowState 
step 100
bowState BOW
bowState 
xLenIndex 1
step 100
bowState BOW
bowState 
step 150
bowState BOW
bowState 
step 300
bowState BOW
bowState 
xLenIndex 2
step 100
bowState BOW
bowState 
step 250
bowState BOW
bowState 
step 500
bowState BOW
bowState 
numwords 20000
xLenIndex 0
step 25
bowState BOW
bowState 
step 50
bowState BOW
bowState 
step 100
bowState BOW
bowState 
xLenIndex 1
step 100
bowState BOW
bowState 
step 150
bowState BOW
bowState 
step 300
bowState BOW
bowState 
xLenIndex 2
step 100
bowState BOW
bowState 
step 250
bowState BOW
bowState 
step 500
bowState BOW